# Information Retrieval - Phase 1: Classical Information Retrieval on the IR2025 Collection


In this project, we build and evaluate a **BM25‐based Elasticsearch index** over the **IR2025** corpus as a baseline before any query expansion.

---
> Maria Schoinaki, BSc Student <br />
> Department of Informatics, Athens University of Economics and Business <br />
> p3210191@aueb.gr <br/><br/>

> Nikos Mitsakis, BSc Student <br />
> Department of Informatics, Athens University of Economics and Business <br />
> p3210122@aueb.gr <br/><br/>

_In this notebook, we will:_
1. Configure and create an Elasticsearch index with a custom analyzer  
2. Ingest the IR2025 documents in bulk  
3. Execute top-k retrieval (k=20,30,50) for each test query  
4. Evaluate using MAP@k and Precision@k (k=5,10,15,20) via pytrec_eval  
5. Summarize and analyze our results  

### Start ElasticSearch manually before running the notebook:
On Windows:
- Make sure you have at least JDK 17
- Open a terminal and execute this (or run it as a Windows service):
```bash
C:\path\to\elasticsearch-8.17.2\bin\elasticsearch.bat
```
- No Greek characters should be present in the path.
- Leave that terminal window open.

- If no password was autogenerated execute this to get one:
```bash
.\bin\elasticsearch-reset-password.bat -u elastic
```

In [79]:
# %pip install -r "..\\requirements.txt"

## 1. Setup & Imports  

**3210122 + 3210191 = 6420313**
- So we get the `trec_covid` IR2025 collection.

Here we import all necessary libraries, set up environment variables, and instantiate the Elasticsearch client.


In [80]:
from collections import Counter
import jsonlines
import json
import csv
import pandas as pd
from tqdm import tqdm
import pytrec_eval
from IPython.display import display

Configuration & Parameters

In [81]:
from dotenv import load_dotenv
import os

# Load .env file from the current directory
load_dotenv("..\\secrets\\secrets.env")

# Access environment variables
es_host = os.getenv("ES_HOST")
es_user = os.getenv("ES_USERNAME")
es_pass = os.getenv("ES_PASSWORD")

Connect to ElasticSearch

In [82]:
from elasticsearch import Elasticsearch

es = Elasticsearch(es_host, basic_auth=(es_user, es_pass))

if es.ping():
    print("✅ Connected to ElasticSearch")
else:
    print("❌ Connection failed")

✅ Connected to ElasticSearch


## 2. Analyzer & Index Creation  
We define a **custom English analyzer** (standard tokenizer + lowercase + stopword removal + Porter stemming) and create the Elasticsearch index with BM25 similarity.  


In [95]:
INDEX_NAME = "ir2025-index"

# Delete the index if it already exists
if es.indices.exists(index=INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
    print(f"✅ Index '{INDEX_NAME}' deleted.")

# Define the settings and mappings for the index
settings = {
    "analysis": {
        "filter": {
            "english_stop": {
                "type": "stop",
                "stopwords": "_english_"
            },
            "english_stemmer": {
                "type": "kstem"
            }
        },
        "analyzer": {
            "custom_english": {
                "type": "custom",
                "tokenizer": "standard",
                "filter": [
                    "lowercase", # Converts all terms to lowercase
                    "english_stop", # Removes English stop words
                    "english_stemmer" # Reduces words to their root form usign kstem
                ]
            }
        }
    }
}

mappings = {
    "properties": {
        "doc_id": {"type": "keyword"},
        "text": {
            "type": "text",
            "analyzer": "custom_english",
            "similarity": "BM25"
        }
    }
}

# Create the index with the specified settings and mappings
es.indices.create(
    index=INDEX_NAME,
    settings=settings,
    mappings=mappings
)
print(f"✅ Index '{INDEX_NAME}' created")

✅ Index 'ir2025-index' deleted.
✅ Index 'ir2025-index' created


## 3. Document Ingestion  
Using the `streaming_bulk` helper, we ingest all IR2025 documents in chunks of 500.  
A progress bar (tqdm) provides real‐time feedback on indexing throughput.

In [96]:
from elasticsearch.helpers import streaming_bulk

# Generator function to yield documents
def generate_documents(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            doc = json.loads(line)
            yield {
                "_index": INDEX_NAME,
                "_id": doc["_id"],
                "_source": {
                    "doc_id": doc["_id"],
                    "text": doc["text"]
                }
            }

# Path to the JSONL file
file_path = "../data/trec-covid/corpus.jsonl"

# Count the total number of documents for the progress bar
with open(file_path, 'r', encoding='utf-8') as f:
    total_docs = sum(1 for _ in f)

# Initialize the progress bar
progress = tqdm(unit="docs", total=total_docs)

successes = 0
for ok, action in streaming_bulk(client=es, actions=generate_documents(file_path), chunk_size=500):
    progress.update(1)
    successes += int(ok)

progress.close()
print(f"✅ Indexed {successes}/{total_docs} documents into '{INDEX_NAME}'")

100%|██████████| 171332/171332 [00:53<00:00, 3191.38docs/s]

✅ Indexed 171332/171332 documents into 'ir2025-index'


## 4. Query Execution  
For each query in `queries.jsonl`, we issue a `match` query on the `text` field and collect the top-k document IDs for k=20,30,50.  
Results are saved as JSON mappings: `{ query_id: [doc1, doc2, …] }`.

In [97]:
def process_queries_phase_1(queries_path):
    # Load queries
    with open(queries_path, 'r', encoding='utf-8') as f:
        queries = [json.loads(line) for line in f]
        print(f"Loaded {len(queries)} queries.")
        
    INDEX_NAME = "ir2025-index"
    k_values = [20, 30, 50] # Number of top documents to retrieve
        
    runs = {f"run_{k}": {} for k in k_values}
    for k in k_values:
        # Prepare output directory
        output_dir = f"../results/phase_1"
        os.makedirs(output_dir, exist_ok=True)
        for query in tqdm(queries, desc=f"Processing Queries for run with k = {k}"):
            qid = query["_id"]
            query_text = query["text"]
            response = es.search(
                index=INDEX_NAME,
                query={"match": {"text": query_text}},
                size=k
            )
            runs[f"run_{k}"][qid] = {hit["_id"]: hit["_score"] for hit in response["hits"]["hits"]}
                
        with open(os.path.join(output_dir, f'retrieval_top_{k}.json'), 'w', encoding='utf-8') as f:
            json.dump(runs[f"run_{k}"], f, ensure_ascii=False, indent=4)
            print(f"✅ Results saved to: ../results/phase_1/retrieval_top_{k}.json")
    
    return runs
    
runs = process_queries_phase_1("../data/trec-covid/queries.jsonl")

Processing Queries for run with k = 20: 100%|██████████| 50/50 [00:02<00:00, 16.99it/s]


✅ Results saved to: ../results/phase_1/retrieval_top_20.json


Processing Queries for run with k = 30: 100%|██████████| 50/50 [00:00<00:00, 75.48it/s] 


✅ Results saved to: ../results/phase_1/retrieval_top_30.json


Processing Queries for run with k = 50: 100%|██████████| 50/50 [00:00<00:00, 56.19it/s]

✅ Results saved to: ../results/phase_1/retrieval_top_50.json


## 5. Load Qrels & Evaluation  
1. **Load** relevance judgments (qrels) from the TSV file into a Python dict.  
2. **Compute** MAP@k and Precision@k (5,10,15,20) using `pytrec_eval`.  
3. **Cross‐validate** with the `trec_eval` binary to ensure correctness.

In [98]:
def load_qrels(qrels_path="../data/trec-covid/qrels/test.tsv"):
    qrels = {}
    with open(qrels_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter='\t')
        for row in reader:
            qid = row['query-id']
            docid = row['corpus-id']
            relevance = int(row['score'])
            qrels.setdefault(qid, {})[docid] = relevance

    relevant_counts = Counter()
    for qid, docs in qrels.items():
        relevant_counts[qid] = sum(1 for rel in docs.values() if rel > 0)
    print("Average number of relevant documents per query:", int(sum(relevant_counts.values()) / len(relevant_counts)))

    return qrels

qrels = load_qrels()

Average number of relevant documents per query: 493


In [99]:
print(pytrec_eval.supported_measures)

{'Rprec', 'infAP', 'bpref', 'relative_P', 'recip_rank', 'gm_map', 'runid', 'num_nonrel_judged_ret', 'set_relative_P', 'utility', 'Rprec_mult', 'num_ret', 'iprec_at_recall', 'success', 'num_rel_ret', 'P', 'gm_bpref', 'Rndcg', 'ndcg_cut', 'set_map', 'recall', 'set_P', 'binG', 'ndcg_rel', 'set_recall', 'num_rel', 'G', 'set_F', 'map_cut', 'map', 'relstring', '11pt_avg', 'ndcg', 'num_q'}


In [100]:
def compute_metrics(qrels, runs, folder, metrics=['map', 'P_5', 'P_10', 'P_15', 'P_20']):    
    # Metrics to Evaluate
    evaluator = pytrec_eval.RelevanceEvaluator(qrels, {'map', 'P'})
    
    for run_name, run in runs.items():
        k = run_name.split("_")[1]
        print(f"Computing metrics for run with k = {k}")
        
        # Verify how many documents were retrieved per query
        # for query_id, docs in run.items():
            # num_docs = len(docs)
            # print(f"Query ID: {query_id} - Retrieved Documents: {num_docs}")
            
        results = evaluator.evaluate(run)
        
        #Print available metrics for debugging
        # first_query = list(results.keys())[0]
        # print(f"Available metrics for {first_query}: {list(results[first_query].keys())}")
        
        # Compute average metrics
        avg_scores = {metric: 0.0 for metric in metrics}
        num_queries = len(results)
        
        for res in results.values():
            for metric in metrics:
                avg_scores[metric] += res.get(metric, 0.0)
        
        for metric in metrics:
            avg_scores[metric] /= num_queries
                                                                                                                                               
        # Prepare output directory
        output_dir = os.path.join("../results", folder)
        os.makedirs(output_dir, exist_ok=True)
        
        # Save per-query metrics
        per_query_path = os.path.join(output_dir, f"per_query_metrics_top_{k}.json")
        with open(per_query_path, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=4)
        
        # Save average metrics
        avg_metrics_path = os.path.join(output_dir, f"average_metrics_top_{k}.json")
        with open(avg_metrics_path, "w", encoding="utf-8") as f:
            json.dump(avg_scores, f, indent=4)
        
        print(f"✅ Per-query metrics saved to: {per_query_path}")
        print(f"✅ Average metrics saved to: {avg_metrics_path}\n")
        
compute_metrics(qrels, runs, "phase_1")

Computing metrics for run with k = 20
✅ Per-query metrics saved to: ../results\phase_1\per_query_metrics_top_20.json
✅ Average metrics saved to: ../results\phase_1\average_metrics_top_20.json

Computing metrics for run with k = 30
✅ Per-query metrics saved to: ../results\phase_1\per_query_metrics_top_30.json
✅ Average metrics saved to: ../results\phase_1\average_metrics_top_30.json

Computing metrics for run with k = 50
✅ Per-query metrics saved to: ../results\phase_1\per_query_metrics_top_50.json
✅ Average metrics saved to: ../results\phase_1\average_metrics_top_50.json



In [101]:
def compare_phases(phases, k_values=[20, 30, 50], metrics=['map', 'P_5', 'P_10', 'P_15', 'P_20']):
    """
    Display and optionally compare retrieval metrics for 1 to 4 phases.
    Parameters:
    - phases: dict mapping phase names to base file paths, e.g.
        {
            "Phase 1": "../results/phase_1/average_metrics_top_{}.json",
            "Phase 2": "../results/phase_2/average_metrics_top_{}.json",
            ...
        }
    - k_values: list of cutoff values to compare (e.g. [20, 30, 50])
    - metrics: list of TREC metric keys (e.g. ['map', 'P_5', 'P_10'])

    Returns:
    - pandas DataFrame with metrics for all phases at each k
    """
    comparison = []

    for k in k_values:
        row = {"k": k}
        for phase_name, base_path in phases.items():
            try:
                with open(base_path.format(k), "r") as f:
                    phase_metrics = json.load(f)
                row[f"{phase_name} MAP"] = phase_metrics["map"]
                for m in metrics[1:]: # exclude MAP
                    row[f"{phase_name} avgPre@{m[2:]}"] = phase_metrics[m]
            except FileNotFoundError:
                print(f"⚠️ File not found: {base_path.format(k)}")
        comparison.append(row)

    df = pd.DataFrame(comparison)
    df.sort_values("k", inplace=True)
    df.set_index("k", inplace=True) # Set 'k' column as the index for visualization purposes
    display(df)
    return df

In [ ]:
phases = {
    "Phase 1": "../results/phase_1/average_metrics_top_{}.json",
    #"Phase 2": "../results/phase_2/average_metrics_top_{}.json",
    #"Phase 3": "../results/phase_3/average_metrics_top_{}.json",
    #"Phase 4": "../results/phase_4/average_metrics_top_{}.json"
}
_ = compare_phases(phases)

,Phase 1 MAP,Phase 1 avgPre@5,Phase 1 avgPre@10,Phase 1 avgPre@15,Phase 1 avgPre@20
k,,,,,
20,0.020569,0.64,0.582,0.564,0.548
30,0.027753,0.64,0.582,0.564,0.549
50,0.039911,0.64,0.582,0.564,0.549
